# Kapitel 3 - Klimawandel

Den Klimawandel an nur einer einzelnen Wetterstation abzulesen ist nicht ganz leicht. Das liegt vor allem an diesen Gründen:

- Die Erde erwärmt sich nicht überall gleich schnell. Und es gibt sogar einige wenige Orte, an denen es über die letzten Jahre eher kälter als wärmer wurde... an diesen Orten kann man also schon prinzipiell keine Erwärmung finden! Das schauen wir uns später nochmal genauer an.
- Im Klimasystem der Erde passiert sehr viel in unterschiedlichen Zeitskalen: Von Regen, der in 15 Minuten wieder vorbei sein kann, über Hoch- und Tiefdruckgebiete, die sich in Tagen bis Wochen verändern, bis zu sehr langsamen Phänomenen: vielleicht hast du schonmal vom El Niño gehört - der kommt in etwa alle 2 bis 7 Jahre. Man muss aufpassen, dass diese sogenannte "Klima-Variabilität" nicht die Daten beeinflusst, die man nutzt um den Klimawandel zu bestimmen! Du hast ja mittlerweile auch schon gesehen, dass sehr viel "Rauschen" in den Daten ist, die Kurven also nicht sehr "glatt" sind.
- Last but not least kann es auch mit der Messstation Probleme geben: Hat sich das Messgerät verändert? Ist die Station umgezogen? Wurde der Boden / die Umgebung um die Station verändert, so dass sich die Gegend um die Station jetzt z.B. unter der Sonne mehr aufwärmt?

Also los geht's: Zuerst müssen wir nochmal die Daten laden.

In [ ]:
# Programmierbibliotheken importieren

import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np

import xarray as xr
import pandas as pd
from matplotlib.patches import Rectangle
from matplotlib.collections import PatchCollection
from matplotlib.colors import ListedColormap
import matplotlib.colors as mcolors
import matplotlib.patches as patches

In [ ]:
d = pd.read_csv(
    "daten/produkt_klima_tag_19340101_20221231_02932.txt",
    sep=";",
    skipinitialspace=True,
)

data = xr.DataArray(d["TMK"], coords={"time": d["MESS_DATUM"]})
data["time"] = pd.to_datetime(data.time, format="%Y%m%d")
data = data.sel(time=slice(np.datetime64("1973-01-01"), None))

In [ ]:
d = pd.read_csv(
    "daten/produkt_klima_tag_19340101_20221231_02932.txt",
    sep=";",
    skipinitialspace=True,
)

data = xr.DataArray(d["TMK"], coords={"time": d["MESS_DATUM"]})
data["time"] = pd.to_datetime(data.time, format="%Y%m%d")

In [ ]:
data = data.sel(time=slice(np.datetime64("1973-01-01"), None))

## 3.1 Temperaturdurchschnitte in Leipzig

Damit der Klimawandel nicht von der "Klimavariabilität" überdeckt wird, können wir uns zunächst mal Jahresdurchschnittswerte anschauen:

In [ ]:
yearly_averages = data.groupby("time.year").mean()

yearly_averages.plot.scatter()
plt.show()

Was siehst du in dem Diagramm?

Antwort: 

Was fällt dir auf? Kannst du einen Trend erkennen?

Antwort:

Der Trend ist nicht ganz eindeutig: vielleicht überrascht es dich, dass es an anderen Stellen klarer sichtbar ist.

Werden also alle hinter's Licht geführt von Klimawissenschaftlern? Nein - Denn worüber Forscher meistens sprechen sind Jahresdurchschnitte für die **gesamte Erde**, wärend wir uns nur die Temperatur in Leipzig angeschaut haben. Und wenn man den Durchschnitt über die gesamte Erde nimmt, "mittelt sich viel Rauschen heraus". Du kannst dir zum Beispiel vorstellen, dass es in einem Jahr mit einem sehr heißen Sommer in Deutschland oft einen kalten Sommer anderswo auf der Welt gibt. 

Wir kommen später nochmal auf den Unterschied von globalen Durchschnitten und dem Leiziger / sächsischen Klima zurück, wenn wir uns "Klimastreifen" anschauen.

## 3.2 Lineare Regression

Du hast vermutlich gesehen, dass es in den Daten der Station Leipzig/Halle tendenziell doch leicht bergauf geht mit der Temperatur. Können wir irgendwie genauer sagen, wie viel das pro Jahr ist?

Um das zu machen schauen wir uns etwas an, was Wissenschaftler:innen "Lineare Regression" nennen. Dabei legt man im einfachsten Fall eine Gerade auf die Daten... und verschiebt die dann so lange hin und her, bis so viele Punkte wie möglich so nah wie möglich an der Geraden dran sind.

Ich hab im nächsten Codeblock mal 3 Geraden definiert - ich denke, man sieht, dass einige davon besser zu den Daten passen als andere:

In [ ]:
def f(x, m, b):
    return m * x + b


yearly_averages = data.groupby("time.year").mean()

yearly_averages.plot.scatter(color="k")

plt.plot(yearly_averages.year, f(yearly_averages.year, 0.1, -190))
plt.plot(yearly_averages.year, f(yearly_averages.year, 2, -4000))
plt.plot(yearly_averages.year, f(yearly_averages.year, -0.3, 610))
plt.ylim(0, 20)
plt.show()

Welche der Geraden passt am besten zu den Leipziger Temperaturdurchschnittswerten?

Antwort: 

Man muss das zum Glück nicht alles von Hand machen - man kann mit Methoden aus der Mathematik definieren, was es bedeutet, dass eine Gerade den "kleinsten Fehler" macht... und dann kann man ausrechnen, welche Gerade die beste ist. Das machen wir (bzw. der Computer) jetzt:

In [ ]:
from scipy import optimize

popt, pcov = optimize.curve_fit(f, yearly_averages.year, yearly_averages)
print(
    "Optimale Steigung: {:.3f}".format(popt[0]),
    "Optimaler y-Achsen-Abschnitt: {:.3f}".format(popt[1]),
)

Schauen wir uns die Gerade an:

In [ ]:
x = yearly_averages.year
y = yearly_averages

plt.scatter(x, y, label="Daten")
plt.plot(
    [min(x), max(x)], [f(min(x), *popt), f(max(x), *popt)], "r--", label="Beste Gerade"
)
plt.legend()

plt.ylim(5, 15)
plt.show()

Das sieht doch gut aus!

Und wenn wir uns die Geradengleichung anschauen können wir auch herausfinden, wie viel es pro Jahr in etwa wärmer wird: Dazu müssen wir auf die Steigung der Geraden schauen:

In [ ]:
"Steigung: {:.3f}".format(popt[0])

Welche Einheiten hat diese Steigung? (Die Antwort ist die Einheit der y-Achse, geteilt durch die Einheit der x-Achse)

Antwort:

Kannst du herausfinden, 1) wie viel es in 10 Jahren wärmer wird? 2) wie lange es etwa dauern sollte, um ein weiteres °C wärmer zu werden?

Antwort 1:

Antwort 2:

Achtung: die Zeit, die wir hier für 1°C zusätzliche Erwärmung ausgerechnet haben muss für die Zukunft nicht richtig sein, nicht einmal für Leipzig! Denn hier haben wir einfach gesagt "die Temperatur folgt über die Zeit dem Verlauf einer Geraden" ohne das irgendwie zu begründen. Das ist eine extrem starke Vereinfachung. Und dazu hängt die zukünftige Klimaveränderung natürlich davon ab, wie viel Treibhausgase ausgestoßen werden - es kann also sein, dass ein zusätzliches °C Erwärmung noch schneller erreicht wird (z.B. wenn wir mehr CO2 ausstoßen), oder dass es deutlich länger dauert (wenn wir den Klimawandel effektiv bekämpfen) - oder dass es überhaupt nie erreicht wird.

## 3.4 Klimastreifen

Die wohl bekannteste Grafik aus der Klimawissenschaft sind die sogenannten "Klimastreifen", die sich der Britische Wissenschaftler Ed Hawkins ausgedacht hat:

![Klimastreifen](klimastreifen.png)

Das hängt mittlerweile auch im "Museum of Modern Art" in New York, man kann Socken mit den Farben kaufen - und die Sachenbrücke im Clara-Zetkin-Park ist auch damit bemalt:

![Klimastreifen Socken](klimastreifen_socken.png) ![Klimastreifen Sachsenbrücke](klimastreifen_sachsenbrücke.png)

Was zeigen die Streifen?
- Jeder Streifen steht für die Weltdurchschnittstemperatur in einem Jahr - links ist früher und rechts ist heute
- Die Farbe der Streifen symbolisiert, wie warm ein Jahr war.
- Um einen Bezugswert für ein "normal warmes" Jahr zu haben, hat sich Ed Hawkins den Durchschnitt der Temperaturen der Jahre von **1961 bis 2010** angeschaut. Alle Jahre, die wärmer als dieser Durchschnitt sind, sind rot, alle die kälter sind sind blau gemalt. Je intensiver die Farbe ist, desto deutlicher unterscheidet sich die Temperatur vom Bezugswert.

Man kann sich diese Streifen auch für die Daten bei sich vor Ort ausrechnen. Das wollen wir im Folgenden machen! Aber dafür sind die Daten der Wetterstation in Leipzig/Halle nicht lang genug - die hatten ja vor 1972 eine Lücke.

Glücklicherweise bietet der Deutsche Wetterdienst die Durchschnittstemperaturen in den einzelnen Bundesländern ab 1881 an.

Beginnen wir wieder damit, die Daten zu laden:

In [ ]:
df = pd.read_csv(
    "daten/regional_averages_tm_year.txt",
    sep=";",
    skipinitialspace=True,
    skiprows=1,
)

Daten gibt's für alle Bundesländer, suche dir eines aus, das dich interessiert, indem du \<durch Bundeslandnamen ersetzen\> durch den Namen eines Bundeslandes ersetzt (wichtig: nur das innerhalb der Anführungszeichen ersetzen!)

Folgende Bundesländer stehen zur Auswahl:

In [ ]:
df.columns.values[2:-1]

In [ ]:
bundesland = "<durch Bundeslandnamen ersetzen>"
zeitreihe_bundesland = xr.DataArray(
    df[bundesland], coords={"year": df["Jahr"]}
).sel(
    year=slice(None, 2024)
)  # hier nehmen wir das Jahr 2025 aus, weil wir dafür noch nicht alle Daten haben - läuft ja schließlich noch!

Als nächstes müssen wir den Bezugswert berechnen, dessen Temperatur darüber entscheidet, was die Grenze zwischen kälteren blauen und wärmeren roten Jahren ist. Erinnerst du dich noch, welche Jahre man dazu benutzt? Ersetze im Folgenden Codeblock ANFANGSJAHR und ENDJAHR entsprechend.

In [ ]:
mittelwert = zeitreihe_bundesland.sel(year=slice(ANFANGSJAHR, ENDJAHR)).mean()

# das brauchen wir später, um zu entscheiden, wie intenstiv rot und blau die Farben sein sollen:
standardabweichung = zeitreihe_bundesland.sel(year=slice(1961, 2010)).std()

Wie warm ist das genau?

In [ ]:
print(f"Bezugswert: {float(mittelwert):.1f}°C")

Jahre, deren Durchnittstemperatur in Sachsen wärmer als dieser Wert sind sind also gleich rot.

Und damit können wir jetzt auch schon die Klimastreifen zeichnen lassen!

In [ ]:
# Erstelle eine neue Grafik mit einer Größe von 15x8 Inches
plt.figure(figsize=(15, 8))
ax = plt.gca()

# Das sorgt dafür, dass die Daten in den richtigen Farben angezeigt werden
norm = mcolors.Normalize(
    vmin=mittelwert - 3 * standardabweichung, vmax=mittelwert + 3 * standardabweichung
)

# Hier definieren wir die Farben, die wir verwenden wollen - im Computer sind Farben durch bestimmte Codes gespeichert, die im Prinzip aussagen, wie viel Rot, Blau und Grün der Computer für die Farbe verwenden soll.
cmap = ListedColormap(
    [
        "#08306b",
        "#08519c",
        "#2171b5",
        "#4292c6",
        "#6baed6",
        "#9ecae1",
        "#c6dbef",
        "#deebf7",
        "#fee0d2",
        "#fcbba1",
        "#fc9272",
        "#fb6a4a",
        "#ef3b2c",
        "#cb181d",
        "#a50f15",
        "#67000d",
    ]
)

# Hier erstellen wir Stück für Stück die Klimastreifen - farbige Rechtecke an der richtigen Position.
for i in range(len(zeitreihe_bundesland.year)):
    # Die Position des Rechtecks entlang der x-Achse hängt vom Jahr ab, für das wir das Rechteck zeichnen wollen.
    x = zeitreihe_bundesland.year[i] - 0.5
    # damit legen wir die Breite und Höhe des Rechtecks fest
    width = 1.001
    height = 1
    # hier bestimmen wir die Farbe, die der Streifen, den wir gerade zeichnen, haben soll.
    color = cmap(norm(zeitreihe_bundesland[i]))
    # Jetzt sind wir mit den Vorbereitungen fertig und können das Rechteck zeichnen.
    rect = patches.Rectangle((x, 0), width, height, facecolor=color, edgecolor="none")
    ax.add_patch(rect)

# Das legt fest, das unsere x-Achse mit dem ersten Jahr im Datensatz beginnen und nach dem letzten Jahr im Datensatz enden soll:
ax.set_xlim(
    zeitreihe_bundesland.year.min() - 0.5, zeitreihe_bundesland.year.max() + 0.5
)

# Das sorgt dafür, dass die y-Achse nicht angezeigt wird
ax.yaxis.set_visible(False)

plt.xlabel("Jahr")
plt.tight_layout()
plt.show()

Damit es noch etwas hübscher aussieht können wir noch die x-Achsenbeschriftung ausblenden - und das Bild dann abspeichern lassen. Nach dem Ausführen dieses Codes sollte ein Bild mit dem Namen `meine_klimastreifen.jpeg` in eurem Ordner auftauchen.

In [ ]:
# Das erstellt eine neue Grafik mit einer Größe von 15x8 Inches:
plt.figure(figsize=(15, 8))
ax = plt.gca()

# Das sorgt dafür, dass die Daten in den richtigen Farben angezeigt werden
norm = mcolors.Normalize(
    vmin=mittelwert - 3 * standardabweichung, vmax=mittelwert + 3 * standardabweichung
)

# Hier definieren wir die Farben, die wir verwenden wollen
# im Computer sind Farben durch bestimmte Codes gespeichert, die im Prinzip aussagen, wie viel Rot, Blau und Grün der Computer für jede Farbe zusammen mischen soll.

cmap = ListedColormap(
    [
        "#08306b",
        "#08519c",
        "#2171b5",
        "#4292c6",
        "#6baed6",
        "#9ecae1",
        "#c6dbef",
        "#deebf7",
        "#fee0d2",
        "#fcbba1",
        "#fc9272",
        "#fb6a4a",
        "#ef3b2c",
        "#cb181d",
        "#a50f15",
        "#67000d",
    ]
)

# Hier erstellen wir Stück für Stück die Klimastreifen - farbige Rechtecke an der richtigen Position.
for i in range(len(zeitreihe_bundesland.year)):
    # Die Position des Rechtecks entlang der x-Achse hängt vom Jahr ab, für das wir das Rechteck zeichnen wollen.
    x = zeitreihe_bundesland.year[i] - 0.5
    # damit legen wir die Breite und Höhe des Rechtecks fest
    width = 1.001
    height = 1
    # hier bestimmen wir die Farbe, die der Streifen, den wir gerade zeichnen, haben soll.
    color = cmap(norm(zeitreihe_bundesland[i]))
    # Jetzt sind wir mit den Vorbereitungen fertig und können das Rechteck zeichnen.
    rect = patches.Rectangle((x, 0), width, height, facecolor=color, edgecolor="none")
    ax.add_patch(rect)

# Das legt fest, das unsere x-Achse mit dem ersten Jahr im Datensatz beginnen und nach dem letzten Jahr im Datensatz enden soll:
ax.set_xlim(
    zeitreihe_bundesland.year.min() - 0.5, zeitreihe_bundesland.year.max() + 0.5
)

plt.axis("off")

# Das sorgt dafür, dass die y-Achse nicht angezeigt wird
ax.yaxis.set_visible(False)

# "Jahr" an die x-Achse schreiben
plt.xlabel("Jahr")

# nur kleine weiße ränder um die Abbildung zulassen
plt.tight_layout()

# bild speichern
plt.savefig("meine_klimastreifen.jpeg", dpi=300, bbox_inches="tight")
plt.show()

Hier zum Vergleich nochmal die Streifen für die gesamte Erde:

![Klimastreifen](klimastreifen.png)

In beiden Fällen erkennt man, dass es mit der Zeit immer röter, also wärmer wird - und das auch in den letzten Jahren immer schneller. Ein Unterschied ist, dass die Farben im Diagram für die gesamte Welt einen "geordneteren" Verlauf nehmen als die regionalen Daten. Kannst du dir denken warum? Erinnere dich an den Anfang des Notebooks und wie die einzelnen Datenpunkte im Zeitverlauf für Leipzig ausgesehen haben. Wenn du dich nicht erinnerst, schau nochmal in den Text danach.

Antwort:

Wenn du dir zum Abschluss noch eine andere coole Animation zu Klimadaten anschauen willst, schau mal [hier](https://svs.gsfc.nasa.gov/5190/). Die Farben in der Animation sind ähnlich wie die der Klimastreifen, der größte Unterschied ist, dass die sich das für einzelne Monate statt für Jahre anschauen.

Es bleibt noch zu sagen, dass Wetterdaten bei weitem nicht der einzige Beweis für den Klimawandel sind. Unter anderem nutzen Forscher auch noch folgendes:

- Bestimmte Umweltprozesse reagieren auf Temperaturschwankungen. Zum Beispiel wachsen Bäume unterschiedlich schnell je nachdem, wie warm es ist. Dadurch kann man dann von der Dicke ihrer Jahresringe auf das Klima zurückschließen. Das ist besonders wertvoll, wenn man das Klima der Zeit studieren will, in der Menschen noch nicht systematisch Wetterdaten aufgezeichnet haben. Das Feld das sich damit beschäftigt nennt sich Paläo-Klimatologie.
- Wir können im Labor messen oder theoretisch bestimmen, welche Wirkung CO2 und andere Treibhausgase auf die Erderwärmung haben. Und das kann man dann mit vielen anderen Dingen, die wir über die Atmosphäre, den Ozean, und andere Aspekte des Erdsystems wissen kombinieren und daraus Computermodelle bauen. Was dabei herauskommt sind Klimamodelle, mit denen wir simulieren können, wie genau sich das Klima verändert, wenn wir als Menschheit in das System eingreifen - etwa indem wir Treibhausgase ausstoßen.